In [1]:
from tpot import TPOTClassifier

from sklearn.metrics import accuracy_score

In [2]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)
import config.settings
import os
import pandas as pd

In [3]:
def read_csv(path:os.path):
    """

    :return:
    """
    df = pd.read_csv((path))
    # train, test =sklearn.model_selection.train_test_split(df,test_size=0.2)

    return df


In [4]:
root = config.settings.get_project_path()
dataset_path = os.path.join(root,'Data')

df= read_csv(os.path.join(dataset_path,'train.csv'))
df_test = read_csv(os.path.join(dataset_path,'test.csv'))

In [5]:
from sklearn.model_selection import train_test_split
X = df.loc[:, df.columns != 'class']

Y = df.loc[:, df.columns == 'class']
test_size = 0.2
shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=test_size, shuffle=shuffle)


In [6]:
tpot_pipeline = TPOTClassifier(generations=32, 
                               population_size=128, 
                               offspring_size=128,
                               scoring='accuracy', 
                               max_time_mins=200,
                               n_jobs=-1,
                               early_stop=52,
                               log_file='./tpot_log_1.txt',
                               subsample=0.5,
                               verbosity=2)
tpot_pipeline.fit(train_X, train_Y)

print("\nTPOT  Accuracy: ", accuracy_score(test_Y, tpot_pipeline.predict(test_X)))

tpot_pipeline.export('tpot_1.py')

/home/sami02/anaconda3/envs/AutoML/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/sami02/anaconda3/envs/AutoML/lib/python3.8/site-packages/tpot/tpot.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pretest_y[0:unique_target_idx.shape[0]] = \
/home/sami02/anaconda3/envs/AutoML/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Optimization Progress:   0%|          | 0/128 [00:00<?, ?pipeline/s]


Best pipeline: GaussianNB(MaxAbsScaler(input_matrix))

TPOT  Accuracy:  0.6974487243621811


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv(os.path.join(dataset_path,'train.csv'))
# features = tpot_data.drop('target', axis=1)
from sklearn.model_selection import train_test_split
X = df.loc[:, df.columns != 'class']

Y = df.loc[:, df.columns == 'class']
test_size = 0.2
shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=test_size, shuffle=shuffle)


# Average CV score on the training set was: 0.6880632210025452
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    GaussianNB()
)

exported_pipeline.fit(train_X, train_Y)
print(exported_pipeline.score(test_X,test_Y))

/home/sami02/anaconda3/envs/AutoML/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7170585292646323


In [10]:
df_test = read_csv(os.path.join(dataset_path,'test.csv'))
test = df_test.loc[:, df_test.columns != 'id']
id_ = df_test.loc[:,df_test.columns == 'id']
test_result = exported_pipeline.predict(test)
id_['Predicted'] = list(test_result)
id_.to_csv('submission.csv',index=False)

/tmp/ipykernel_101049/3698042793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_['Predicted'] = list(test_result)
